In [137]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [138]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [139]:
#Mapping months to numeric values
months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'June','07':'July','08':'Aug','09':'Sept','10':'Oct','11':'Nov','12':'Dec'}

In [140]:
import pandas as pd

#Importing files from specific folders and list of holidays
bus_no='6080'                                                   #Route number - Change this value accordingly 
path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Up'   #Up Direction files folder Path - Change the path accordingly    
down_path = r'D:/Dharmit/Final Year Project/Clean data Vikhroli/'+bus_no+'/Down' #Down Direction files folder path - Change the path accordingly
allfiles = glob.glob(path + "/*.csv")
down_allfiles = glob.glob(down_path + "/*.csv")
Holiday_colnames = ['Holiday', 'Date', 'Day', 'Month']
Holiday_data = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays.csv', names=Holiday_colnames)     #Holiday list for 2018
Holiday_2017 = pd.read_csv('D:\Dharmit\Final Year Project\List of holidays 2017.csv', names=Holiday_colnames)      #Holiday list for 2017
feature_names =  ['Time Slot','Trips','Passengers','Day','isHoliday']

# Up Direction Processing

In [141]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

#Dicing only requird data from list of holidays
holiday_day = Holiday_data['Day']
holiday_month = Holiday_data['Month']
holiday_day_2017 = Holiday_2017['Day']
holiday_month_2017 = Holiday_2017['Month']

In [142]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,23):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][29:]
    date = os.path.splitext(base)[0][26:28]

    month = os.path.splitext(base)[0][23:25]
    year = os.path.splitext(base)[0][18:22]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [144]:
from sklearn import svm, datasets
C=1.0
trips=[]
up_pass=[]
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    up_pass.append(round(pass_pred))
    target=np.append(round(pass_pred),target)
    print target
    
    X=df[j][feature_names].drop(['Trips','Time Slot'], axis=1).values
    y=df[j]['Trips'].values
    y=y.astype('int')
    if(j==19  or len(np.unique(y))<2):
        predicted_trips=0
    else:
        svc = svm.SVC(kernel='rbf',degree=3, C=C,).fit(X, y)
        target=target.reshape(1,3)
        predicted_trips=svc.predict(target)
        
    trips.append(int(predicted_trips))
    print('Predicted trips for timeslot: %d is: %d' % (j+1,predicted_trips))
    

for i in range(len(trips)):
        if trips[i]=='':
            trips[i]=0   
trips = [0 if x is None else x for x in trips]

Timeslot 1 :
[ 9.  4.  0.]
Predicted trips for timeslot: 1 is: 0
Timeslot 2 :
[ 4.  4.  0.]
Predicted trips for timeslot: 2 is: 1
Timeslot 3 :
[ 15.   4.   0.]
Predicted trips for timeslot: 3 is: 5
Timeslot 4 :
[ 32.   4.   0.]
Predicted trips for timeslot: 4 is: 6
Timeslot 5 :
[ 32.   4.   0.]
Predicted trips for timeslot: 5 is: 5
Timeslot 6 :
[ 36.   4.   0.]
Predicted trips for timeslot: 6 is: 3
Timeslot 7 :
[ 23.   4.   0.]
Predicted trips for timeslot: 7 is: 3
Timeslot 8 :
[ 31.   4.   0.]
Predicted trips for timeslot: 8 is: 2
Timeslot 9 :
[ 34.   4.   0.]
Predicted trips for timeslot: 9 is: 2
Timeslot 10 :
[ 33.   4.   0.]
Predicted trips for timeslot: 10 is: 2
Timeslot 11 :
[ 33.   4.   0.]
Predicted trips for timeslot: 11 is: 3
Timeslot 12 :
[ 47.   4.   0.]
Predicted trips for timeslot: 12 is: 4
Timeslot 13 :
[ 58.   4.   0.]
Predicted trips for timeslot: 13 is: 3
Timeslot 14 :
[ 81.   4.   0.]
Predicted trips for timeslot: 14 is: 4
Timeslot 15 :
[ 110.    4.    0.]
Predicted 

In [145]:
print trips

[0, 1, 5, 6, 5, 3, 3, 2, 2, 2, 3, 4, 3, 4, 3, 4, 3, 3, 4, 0]


# Down direction processing

In [146]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df2=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df3=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df4=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df5=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df6=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df7=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df8=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df9=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df10=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df11=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df12=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df13=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df14=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df15=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df16=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df17=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df18=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df19=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df20=pd.DataFrame(columns=['Time Slot','Trips','Passengers','Day','isHoliday'])
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]

In [147]:
Timeslot_values= ['00:00:01-05:00:00','05:00:01-06:00:00','06:00:01-07:00:00','07:00:01-08:00:00','08:00:01-09:00:00','09:00:01-10:00:00','10:00:01-11:00:00','11:00:01-12:00:00','12:00:01-13:00:00','13:00:01-14:00:00','14:00:01-15:00:00','15:00:01-16:00:00','16:00:01-17:00:00','17:00:01-18:00:00','18:00:01-19:00:00','19:00:01-20:00:00','20:00:01-21:00:00','21:00:01-22:00:00','22:00:01-23:00:00','23:00:01-24:00:00']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

#Obtaining the day and date details for current day 
now = datetime.datetime.now()   
current_month = months[str(now.month).zfill(2)] 
current_day = now.day
isCurrDay_holiday = 0
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week
for i in range(1,24):                            #checking if there is holiday today
    if(holiday_day[i]==current_day and current_month==holiday_month[i]):
        isCurrDay_holiday=1

for files in down_allfiles:
    #Obtaining Day and date details from filename
    base = os.path.basename(files)
    day = os.path.splitext(base)[0][31:]
    date = os.path.splitext(base)[0][29:31]
    month = os.path.splitext(base)[0][26:28]
    
    #Determining whether holiday or not
    year = os.path.splitext(base)[0][21:25]
    #Determining whether holiday or not
    holiday = 0
    if year=='2017':
        for i in range(1,22):
            if(str(holiday_day_2017[i]).zfill(2)==date and months[month]==holiday_month_2017[i]):
            #print'Holiday 2017'
                holiday=1
    else:
        for i in range(1,23):
            if(holiday_day[i]==date and months[month]==holiday_month[i]):
                holiday=1
            
    #Pre-processing data into required format
    day = Days[day]                                        #Mapping day to numeric value
    with open(files,'rb') as f:
        timeslot = []
        trips = []
        passengers = []
        reader = csv.reader(f, delimiter=",")
        rows = [r for r in reader]
        for x in rows[1:]:
            timeslot.append(x[1])
            trips.append(x[2])
            if(x[3]==''):
                x[3]=0
            passengers.append(x[3])
 
    #Adding data into the dataframes created previously according to time slots
    for j in xrange(len(timeslot)):
        df[j]= df[j].append(pd.Series([timeslot[j],trips[j],int(passengers[j]),day,holiday], index = ['Time Slot','Trips','Passengers','Day','isHoliday']), ignore_index=True)
        df[j]=df[j].replace('',0)                      #Replacing NAN values by 0

In [148]:
from sklearn import svm, datasets
C=1.0
down_trips=[]
down_pass=[]
for j in xrange(len(timeslot)):
    print 'Timeslot',j+1,':'
    pass_data=np.asarray(df[j][['Day','isHoliday']])
    pass_labels=np.asarray(df[j]['Passengers'])
    target=[[current_dow,isCurrDay_holiday]]                      # Example target list to check output format and debugging

    passenger_predictor = LinearRegression(normalize=True)          #Linear regression model to predict passenger frequency
    passenger_predictor.fit(pass_data, pass_labels)
    #err_measure = passenger_predictor.score(pass_data, pass_labels, sample_weight=None)  #Error in passenger frequency prediction
    pass_pred=passenger_predictor.predict(target)
    down_pass.append(round(pass_pred))
    target=np.append(round(pass_pred),target)
    print target
    
    X=df[j][feature_names].drop(['Trips','Time Slot'], axis=1).values
    y=df[j]['Trips'].values
    y=y.astype('int')
    if(j==19 or len(np.unique(y))<2):
        predicted_trips=0
    else:
        svc = svm.SVC(kernel='rbf',degree=3, C=C,).fit(X, y)
        target=target.reshape(1,3)
        predicted_trips=svc.predict(target)
    down_trips.append(int(predicted_trips))
    print('Predicted trips for timeslot: %d is: %d' % (j+1,predicted_trips))
    
for i in range(len(down_trips)):
        if down_trips[i]=='':
            down_trips[i]=0   
down_trips = [0 if x is None else x for x in down_trips]

Timeslot 1 :
[ 11.   4.   0.]
Predicted trips for timeslot: 1 is: 0
Timeslot 2 :
[ 21.   4.   0.]
Predicted trips for timeslot: 2 is: 2
Timeslot 3 :
[ 79.   4.   0.]
Predicted trips for timeslot: 3 is: 5
Timeslot 4 :
[ 128.    4.    0.]
Predicted trips for timeslot: 4 is: 7
Timeslot 5 :
[ 127.    4.    0.]
Predicted trips for timeslot: 5 is: 4
Timeslot 6 :
[ 119.    4.    0.]
Predicted trips for timeslot: 6 is: 4
Timeslot 7 :
[ 80.   4.   0.]
Predicted trips for timeslot: 7 is: 4
Timeslot 8 :
[ 65.   4.   0.]
Predicted trips for timeslot: 8 is: 4
Timeslot 9 :
[ 48.   4.   0.]
Predicted trips for timeslot: 9 is: 4
Timeslot 10 :
[ 37.   4.   0.]
Predicted trips for timeslot: 10 is: 3
Timeslot 11 :
[ 30.   4.   0.]
Predicted trips for timeslot: 11 is: 4
Timeslot 12 :
[ 23.   4.   0.]
Predicted trips for timeslot: 12 is: 4
Timeslot 13 :
[ 25.   4.   0.]
Predicted trips for timeslot: 13 is: 3
Timeslot 14 :
[ 29.   4.   0.]
Predicted trips for timeslot: 14 is: 3
Timeslot 15 :
[ 31.   4.   0.

In [150]:
print len(trips)
print len(down_trips)

20
20


In [149]:
#Saving Results into CSV files 
forecasted_trips = pd.DataFrame(columns=['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass'])
save_path=r'D:/Dharmit/Final Year Project/Forecasted Trips/'
for j in xrange(len(timeslot)):
    forecasted_trips= forecasted_trips.append(pd.Series([timeslot[j],trips[j],up_pass[j],down_trips[j],down_pass[j]], index = ['Time Slot','Up_Trips','Up_pass','Down_Trips','Down_pass']), ignore_index=True)
filename ='Forecasted_trips_'+bus_no+'_'+str(now.month).zfill(2) +'_'+str(now.day)+'_'+str(day)+'_SVM'
forecasted_trips.to_csv(save_path + filename +'.csv')